## Data Cleaning & Preprocessing: Currency Data
Author: Jingyuan\
Date: 2022-06-05\
Data storage: GCP bucket\
Computational platform: GCP (Google Collab)

### Import Python Libraries

#### Update packages

In [1]:
!pip3 install --upgrade pandas

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!pip3 install --upgrade numpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


#### Import

In [3]:
import pandas as pd
import numpy as np
import os

In [4]:
os.getcwd()

'/content'

### Data Aquisation

In [5]:
! curl -O https://storage.googleapis.com/data_bucket_mma/data/Example_Data.xlsx

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 33824  100 33824    0     0  1270k      0 --:--:-- --:--:-- --:--:-- 1321k


In [6]:
from pathlib import Path
file_path = Path(os.getcwd()+'/Example_Data.xlsx')
file_extension = file_path.suffix.lower()[1:]

In [7]:
print(file_extension)

xlsx


In [8]:
raw_data = pd.read_excel(file_path, sheet_name=None, engine='openpyxl')

In [9]:
raw_data.keys()

dict_keys(['Example_Data', 'Example_DB', 'Example_Answer'])

In [10]:
Example_Answer = raw_data['Example_Answer']
Example_DB = raw_data['Example_DB']
Example_Data = raw_data['Example_Data']

In [11]:
del raw_data

### Data Cleaning and Preprocessing


In [12]:
Example_Data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112 entries, 0 to 111
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Company Info    112 non-null    object
 1   Unnamed: 1      112 non-null    object
 2   Unnamed: 2      112 non-null    object
 3   Unnamed: 3      112 non-null    object
 4   Unnamed: 4      112 non-null    object
 5   Unnamed: 5      112 non-null    object
 6   Company Metric  112 non-null    object
 7   Unnamed: 7      60 non-null     object
 8   Unnamed: 8      112 non-null    object
 9   Unnamed: 9      112 non-null    object
 10  Unnamed: 10     112 non-null    object
dtypes: object(11)
memory usage: 9.8+ KB


In [13]:
new_header = Example_Data.iloc[0].values
Example_Data = Example_Data.iloc[1:,:]
Example_Data.columns = new_header
Example_Data.head()

,Company ID,Company Name,Fiscal Year,Industry,SIC Code,Trading Currency,SP,CDS,APD,ARD,ADA
1,4993687,A Company,2010,Machinery,6085,USD,75,564,109,44,0
2,4993687,A Company,2011,Machinery,6085,USD,66,721,102,47,0
3,4993687,A Company,2012,Machinery,6085,USD,29,1048,97,45,0
4,4993687,A Company,2013,Machinery,6085,USD,56,914,96,40,0
5,4993687,A Company,2014,Machinery,6085,3$,55,950,91,34,0


In [14]:
Example_Data.columns

Index(['Company ID', 'Company Name', 'Fiscal Year', 'Industry', 'SIC Code',
       'Trading Currency', 'SP', 'CDS', 'APD', 'ARD', 'ADA'],
      dtype='object')

In [15]:
Example_Data.dtypes

Company ID          object
Company Name        object
Fiscal Year         object
Industry            object
SIC Code            object
Trading Currency    object
SP                  object
CDS                 object
APD                 object
ARD                 object
ADA                 object
dtype: object

In [16]:
company_info = list(Example_Data.columns[:6])
matric_info = list(Example_Data.columns[6:11])

#### Fix data types
Given that company information could be strings for later clean up, matrics should be integer columns.

In [17]:
np.all(Example_Data[matric_info].fillna(0).astype("str").apply(lambda x: x.str.isdigit()))

True

We notice that matrics columns are numerical, but null values exist. Convert these 5 columns with integer type 'Int64'.

In [18]:
Example_Data[matric_info] = Example_Data[matric_info].astype("Int64")

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:3641: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [19]:
Example_Data.dtypes

Company ID          object
Company Name        object
Fiscal Year         object
Industry            object
SIC Code            object
Trading Currency    object
SP                   Int64
CDS                  Int64
APD                  Int64
ARD                  Int64
ADA                  Int64
dtype: object

In [20]:
Example_Data.describe(include='all')

,Company ID,Company Name,Fiscal Year,Industry,SIC Code,Trading Currency,SP,CDS,APD,ARD,ADA
count,111.0,111,111.0,111,111.0,111,111.000000,59.000000,111.000000,111.000000,111.000000
unique,14.0,15,13.0,4,16.0,3,NaN,NaN,NaN,NaN,NaN
top,4991368.0,D Company,2012.0,Machinery,6085.0,USD,NaN,NaN,NaN,NaN,NaN
freq,10.0,20,13.0,66,15.0,100,NaN,NaN,NaN,NaN,NaN
mean,NaN,NaN,NaN,NaN,NaN,NaN,92.144144,293.305085,86.900901,80.837838,0.234234
std,NaN,NaN,NaN,NaN,NaN,NaN,65.770655,292.946697,40.086034,41.616112,0.571361
min,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,51.000000,37.000000,22.000000,0.000000
25%,NaN,NaN,NaN,NaN,NaN,NaN,44.500000,103.000000,65.000000,66.500000,0.000000
50%,NaN,NaN,NaN,NaN,NaN,NaN,68.000000,149.000000,84.000000,81.000000,0.000000
75%,NaN,NaN,NaN,NaN,NaN,NaN,123.000000,349.500000,98.000000,91.000000,0.000000


#### Trim space for string columns

In [21]:
Example_Data[company_info] = Example_Data[company_info].astype("str").apply(lambda x: x.str.strip())

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:3641: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [22]:
Example_Data.describe(include='all')

,Company ID,Company Name,Fiscal Year,Industry,SIC Code,Trading Currency,SP,CDS,APD,ARD,ADA
count,111,111,111,111,111,111,111.000000,59.000000,111.000000,111.000000,111.000000
unique,14,15,13,4,16,3,NaN,NaN,NaN,NaN,NaN
top,4991368,D Company,2012,Machinery,6085,USD,NaN,NaN,NaN,NaN,NaN
freq,10,20,13,66,15,100,NaN,NaN,NaN,NaN,NaN
mean,NaN,NaN,NaN,NaN,NaN,NaN,92.144144,293.305085,86.900901,80.837838,0.234234
std,NaN,NaN,NaN,NaN,NaN,NaN,65.770655,292.946697,40.086034,41.616112,0.571361
min,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,51.000000,37.000000,22.000000,0.000000
25%,NaN,NaN,NaN,NaN,NaN,NaN,44.500000,103.000000,65.000000,66.500000,0.000000
50%,NaN,NaN,NaN,NaN,NaN,NaN,68.000000,149.000000,84.000000,81.000000,0.000000
75%,NaN,NaN,NaN,NaN,NaN,NaN,123.000000,349.500000,98.000000,91.000000,0.000000


#### Company ID vs Company Name

In [23]:
print("# of unique company IDs:", len(np.unique(Example_Data['Company ID'])))
print("# of unique company:", len(np.unique(Example_Data['Company Name'])))

# of unique company IDs: 14
# of unique company: 15


Company ID perspective

In [24]:
company_id_name_count = (Example_Data
 .groupby(['Company ID','Company Name'])
 .size()
 .reset_index()[['Company ID','Company Name']]
 .sort_values(by='Company ID')
 .groupby('Company ID')
 .size())

In [25]:
ID_flg = company_id_name_count[np.where(company_id_name_count>1)[0]].reset_index()['Company ID'].values
print(ID_flg)

['4976235' '4991368']


ID '4976235' and '4991368' have more than 1 company name pointing at the same ID.

Company name perspective

In [26]:
company_name_id_count = (Example_Data
 .groupby(['Company ID','Company Name'])
 .size()
 .reset_index()[['Company ID','Company Name']]
 .sort_values(by='Company Name')
 .groupby('Company Name')
 .size())

In [27]:
name_flg = company_name_id_count[np.where(company_name_id_count>1)[0]].reset_index()['Company Name'].values
print(name_flg)

['D Company']


Take out problematic IDs and names, ensure the remaining company ID and names are correct.

In [28]:
company_id_name_lookup = (Example_Data
 .groupby(['Company ID','Company Name','Industry'])
 .size()
 .reset_index()[['Company ID','Company Name','Industry']])

company_id_name_lookup_check = company_id_name_lookup[~(company_id_name_lookup['Company ID'].isin(ID_flg) | company_id_name_lookup['Company Name'].isin(name_flg))]
company_id_name_lookup_flg = company_id_name_lookup[company_id_name_lookup['Company ID'].isin(ID_flg) | company_id_name_lookup['Company Name'].isin(name_flg)]

In [29]:
print("Original lookup shape:",company_id_name_lookup.shape)
print("Comany IDs & names for check shape:",company_id_name_lookup_check.shape)
print("Problematic Comany IDs & names shape:",company_id_name_lookup_flg.shape)

Original lookup shape: (16, 3)
Comany IDs & names for check shape: (10, 3)
Problematic Comany IDs & names shape: (6, 3)


Introduce data from database for lookup reference. By using the company ID as key to find company names from db, check if names are consistent between new data and data in the database.

In [30]:
db_company_id_name_lookup = Example_DB.groupby(['Company ID','Company Name','Industry']).size().reset_index()[['Company ID','Company Name','Industry']].astype(str)

In [31]:
db_company_id_name_lookup.sort_values(by="Company Name")

,Company ID,Company Name,Industry
6,4993687,A Company,Machinery
5,4989787,C Company,Machinery
2,500295,D Company,Aerospace and Defense
8,4994275,D Company,Machinery
7,4994208,F Company,Machinery
1,499609,G Company,Auto Components
10,4997627,H Company,Machinery
3,4977221,I Company,Building Products
0,498856,J Company,Auto Components
9,4996197,K Company,Machinery


Note that D Company is corresponding to 2 IDs based on 2 industries.

In [32]:
check_data_lookup = pd.merge(company_id_name_lookup_check,db_company_id_name_lookup,on=['Company ID'],how="left", suffixes=('_data', '_db'))
check_data_lookup

,Company ID,Company Name_data,Industry_data,Company Name_db,Industry_db
0,4977221,I Company,Building Products,I Company,Building Products
1,4986826,L Company,Auto Components,L Company,Auto Components
2,498856,J Company,Auto Components,J Company,Auto Components
3,4989787,C Company,Machinery,C Company,Machinery
4,4993687,A Company,Machinery,A Company,Machinery
5,4994208,F Company,Machinery,F Company,Machinery
6,499609,G Company,Auto Components,G Company,Auto Components
7,4996197,K Company,Machinery,K Company,Machinery
8,4997627,H Company,Machinery,H Company,Machinery
9,5081187,M Company,Machinery,M Company,Machinery


In [33]:
np.all([check_data_lookup['Company Name_data'] == check_data_lookup['Company Name_db']])

True

In [34]:
np.all([check_data_lookup['Industry_data'] == check_data_lookup['Industry_db']])

True

We can confirm that company IDs and names for data that did not signal problems are consistent with our desired data format.

Now fix the problems with partial data signals issues previous.

##### Issue 1: Multiple names to 1 ID

In [35]:
ID_problem = company_id_name_lookup_flg[company_id_name_lookup_flg['Company ID'].isin(ID_flg)]
name_problem = company_id_name_lookup_flg[company_id_name_lookup_flg['Company Name'].isin(name_flg)]

In [36]:
pd.merge(ID_problem,db_company_id_name_lookup, on='Company ID',how="left", suffixes=('_data', '_db'))

,Company ID,Company Name_data,Industry_data,Company Name_db,Industry_db
0,4976235,E Company,Machinery,NaN,NaN
1,4976235,Evil Company,Machinery,NaN,NaN
2,4991368,B Company,Auto Components,NaN,NaN
3,4991368,Banana Company,Auto Components,NaN,NaN


Both ID could not be searched in database. In this case to make sure these records are new input, search based on company names.

In [37]:
pd.merge(ID_problem,db_company_id_name_lookup, on='Company Name',how="left", suffixes=('_data', '_db'))

,Company ID_data,Company Name,Industry_data,Company ID_db,Industry_db
0,4976235,E Company,Machinery,NaN,NaN
1,4976235,Evil Company,Machinery,NaN,NaN
2,4991368,B Company,Auto Components,NaN,NaN
3,4991368,Banana Company,Auto Components,NaN,NaN


We can confirm that these records are new input data. Based on format we observed in database, we take 'E Company' and 'B Company' as corrected values.

In [38]:
ID_problem_corrected = ID_problem.sort_values(by='Company Name').groupby('Company ID').first().reset_index()

In [39]:
ID_problem_corrected 

,Company ID,Company Name,Industry
0,4976235,E Company,Machinery
1,4991368,B Company,Auto Components


##### Issue 2: Multiple ID to 1 name

This problem needs to take multiple steps to fix. Company D in database as reference actually has 2 IDs associated to it, corresponding to its presentation in 2 industries.

In [40]:
db_company_id_name_lookup[db_company_id_name_lookup['Company Name'].isin(name_flg)]

,Company ID,Company Name,Industry
2,500295,D Company,Aerospace and Defense
8,4994275,D Company,Machinery


Based on company name and industry, check if IDs are correct.

In [41]:
check_ids_name = pd.merge(name_problem,db_company_id_name_lookup, on=['Company Name','Industry'],how="left", suffixes=('_data', '_db'))
check_ids_name

,Company ID_data,Company Name,Industry,Company ID_db
0,4994275,D Company,Machinery,4994275
1,500295,D Company,Aerospace and Defense,500295


In [42]:
np.all([check_ids_name['Company ID_data']==check_ids_name['Company ID_db']])

True

IDs are correct, no need to fix IDs for D Company.

**Note:**\
Based on request, if there is ID to many names (fixed) or 1 name to many IDs, ID/name is invalid and need to be cleaned up.

We have performed checks on example data and applied fixes if accapable. Potentially data related to D Company might have ID issues as well and need further investigation, since DB format is the reference for data cleaning, we assume Company D's situation where it has 2 IDs for 2 industries is correct.

Potentially, these records could be dropped.

Now perform fixes accordingly to records that has ID problems (1 ID corresponding to more than 1 company name)

In [43]:
ID_problem_corrected

,Company ID,Company Name,Industry
0,4976235,E Company,Machinery
1,4991368,B Company,Auto Components


In [44]:
cleaned_data = pd.merge(Example_Data,ID_problem_corrected,on=['Company ID','Industry'],how="left") 
cleaned_data.head()

,Company ID,Company Name_x,Fiscal Year,Industry,SIC Code,Trading Currency,SP,CDS,APD,ARD,ADA,Company Name_y
0,4993687,A Company,2010,Machinery,6085,USD,75,564,109,44,0,NaN
1,4993687,A Company,2011,Machinery,6085,USD,66,721,102,47,0,NaN
2,4993687,A Company,2012,Machinery,6085,USD,29,1048,97,45,0,NaN
3,4993687,A Company,2013,Machinery,6085,USD,56,914,96,40,0,NaN
4,4993687,A Company,2014,Machinery,6085,3$,55,950,91,34,0,NaN


In [45]:
cleaned_data['Company Name'] = np.where(cleaned_data['Company ID'].isin(ID_flg), cleaned_data['Company Name_y'],cleaned_data['Company Name_x'])

In [46]:
cleaned_data = cleaned_data.drop(['Company Name_x','Company Name_y'],axis=1)

In [47]:
cleaned_data[cleaned_data['Company ID'].isin(ID_flg)]

,Company ID,Fiscal Year,Industry,SIC Code,Trading Currency,SP,CDS,APD,ARD,ADA,Company Name
9,4991368,2009,Auto Components,12,USD,22,663,110,88,1,B Company
10,4991368,2010,Auto Components,4937,USD,18,862,96,79,0,B Company
11,4991368,2011,Auto Components,4937,USD,22,991,98,69,0,B Company
12,4991368,2012,Auto Components,4937,USD,21,874,119,73,0,B Company
13,4991368,2013,Auto Components,4937,USD,37,376,121,73,0,B Company
14,4991368,2014,Auto Components,2937,USD,44,323,124,71,0,B Company
15,4991368,2015,Auto Components,4937,USD,51,263,135,71,1,B Company
16,4991368,2016,Auto Components,4937,USD,48,247,138,71,1,B Company
17,4991368,2017,Auto Components,4937,USD,50,187,131,70,1,B Company
18,4991368,2018,Auto Components,4937,USD,32,472,135,74,1,B Company


In [48]:
cleaned_data.dtypes

Company ID          object
Fiscal Year         object
Industry            object
SIC Code            object
Trading Currency    object
SP                   Int64
CDS                  Int64
APD                  Int64
ARD                  Int64
ADA                  Int64
Company Name        object
dtype: object

#### Fiscal Year Clean Up

Check if strings are digital values.

In [49]:
cleaned_data['Fiscal Year'][~cleaned_data['Fiscal Year'].str.isdigit()]

46    2Q16
Name: Fiscal Year, dtype: object

In year '2Q16', Q should be replaced by 0.

Check if any fiscal year value falls out of the expected range.

In [50]:
cleaned_data['Fiscal Year'] = np.where(cleaned_data['Fiscal Year']=='2Q16','2016',cleaned_data['Fiscal Year']).astype(int)

In [51]:
years = np.unique(cleaned_data['Fiscal Year']).astype('int')
print(years[~((years>=1999)&(years<=2021))])

[213]


There is a missing 0 in year '213'.  

In [52]:
cleaned_data['Fiscal Year'] = np.where(cleaned_data['Fiscal Year']==213,
                                       2013,
                                       cleaned_data['Fiscal Year'])

In [53]:
print(len(cleaned_data['Fiscal Year'][~((cleaned_data['Fiscal Year']>=1999)&(cleaned_data['Fiscal Year']<=2021))]))

0


In [54]:
np.unique(cleaned_data['Fiscal Year'])

array([2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019])

#### Check SIC Code

In [55]:
SICCode = cleaned_data['SIC Code']

Check if there's any non-4-digit entries 

In [56]:
SIC_flg = SICCode[(SICCode.str.len()!=4)|(~SICCode.str.isdigit())]

In [57]:
cleaned_data[cleaned_data['SIC Code'].isin(SIC_flg)]

,Company ID,Fiscal Year,Industry,SIC Code,Trading Currency,SP,CDS,APD,ARD,ADA,Company Name
9,4991368,2009,Auto Components,12,USD,22,663,110,88,1,B Company
79,4977221,2018,Building Products,Y8,USD,45,133,86,76,0,I Company
87,4996197,2015,Machinery,578B,USD,29,<NA>,101,102,0,K Company


In [58]:
db_lookup_sic = (Example_DB
 .groupby(['Company ID','Company Name','Industry','SIC Code'])
 .size()
 .reset_index()[['Company ID','Company Name','Industry','SIC Code']]
 .astype("str"))

db_lookup_sic.sort_values(by='SIC Code')

,Company ID,Company Name,Industry,SIC Code
1,499609,G Company,Auto Components,4150
3,4977221,I Company,Building Products,5624
8,4994275,D Company,Machinery,5722
5,4989787,C Company,Machinery,5755
10,4997627,H Company,Machinery,5776
9,4996197,K Company,Machinery,5788
11,5081187,M Company,Machinery,5886
6,4993687,A Company,Machinery,6085
7,4994208,F Company,Machinery,6085
0,498856,J Company,Auto Components,6090


From the database, we notice that SIC code has a one-to-many relationship with company ID. Now check example data.

In [59]:
(cleaned_data
 .groupby(['Company ID','Company Name','Industry','SIC Code'])
 .size()
 .reset_index()[['Company ID','Company Name','Industry','SIC Code']]
 .astype("str"))

,Company ID,Company Name,Industry,SIC Code
0,4976235,E Company,Machinery,5878
1,4977221,I Company,Building Products,5624
2,4977221,I Company,Building Products,Y8
3,4986826,L Company,Auto Components,6090
4,498856,J Company,Auto Components,6090
5,4989787,C Company,Machinery,5755
6,4991368,B Company,Auto Components,12
7,4991368,B Company,Auto Components,2937
8,4991368,B Company,Auto Components,4937
9,4993687,A Company,Machinery,6085


There are 3 rows that we flagged previously did not match the formatting requirement.

In [60]:
sic_fix = pd.merge(cleaned_data[cleaned_data['SIC Code'].isin(SIC_flg)],
                   db_lookup_sic,
                   on=['Company ID','Company Name','Industry'],
                   how="left")
sic_fix['SIC Code'] = sic_fix['SIC Code_y']
sic_fix = sic_fix.drop(['SIC Code_x','SIC Code_y'],axis=1)
sic_fix

,Company ID,Fiscal Year,Industry,Trading Currency,SP,CDS,APD,ARD,ADA,Company Name,SIC Code
0,4991368,2009,Auto Components,USD,22,663,110,88,1,B Company,NaN
1,4977221,2018,Building Products,USD,45,133,86,76,0,I Company,5624
2,4996197,2015,Machinery,USD,29,<NA>,101,102,0,K Company,5788


We can find corresponding correct SIC code for I and K Company in database, but as new input data B Company, we need to figure out which SIC code to use.

Usually confirmation on SIC code would go back to the data source provider. Here we assume the SIC code that appears the most is the correct SIC code for B Company.

In [61]:
cleaned_data[cleaned_data['Company Name']=='B Company'].groupby('SIC Code').size()

SIC Code
12      1
2937    1
4937    8
dtype: int64

Based on our previous assumption, in order to fill in irregular values, we choose the SIC code that:
- match with the 4-digit requirement
- the most common value used


In [62]:
BCompanySIC = (cleaned_data[cleaned_data['Company Name']=='B Company']
               .groupby('SIC Code')
               .size().reset_index()
               .sort_values(by=0,ascending=False)
               .iloc[0,0])

In [63]:
print(BCompanySIC)

4937


In [64]:
sic_fix.loc[sic_fix['Company Name'] == 'B Company', 'SIC Code'] = BCompanySIC

In [65]:
sic_fix

,Company ID,Fiscal Year,Industry,Trading Currency,SP,CDS,APD,ARD,ADA,Company Name,SIC Code
0,4991368,2009,Auto Components,USD,22,663,110,88,1,B Company,4937
1,4977221,2018,Building Products,USD,45,133,86,76,0,I Company,5624
2,4996197,2015,Machinery,USD,29,<NA>,101,102,0,K Company,5788


Apply the fix to data. **Assume that 1 company must have the same SIC code, and one SIC code could be corresponding to multiple companies.**

i.e. B Company would have 1 SIC code: 4937

In [66]:
cleaned_fix = pd.merge(cleaned_data,
                       sic_fix[['Company ID','Company Name','Industry','SIC Code']],
                       on=['Company ID','Company Name','Industry'],
                       how="left")

cleaned_fix['SIC Code'] = np.where(cleaned_fix['SIC Code_y'].isnull(), cleaned_fix['SIC Code_x'],cleaned_fix['SIC Code_y'])

In [67]:
np.unique(cleaned_fix['SIC Code'])

array(['4150', '4937', '5624', '5722', '5755', '5776', '5788', '5878',
       '5886', '6085', '6090', '6101'], dtype=object)

In [68]:
cleaned_data = cleaned_fix.drop(['SIC Code_x','SIC Code_y'],axis=1).copy()
del cleaned_fix

Final check on SIC code

In [69]:
(cleaned_data
 .groupby(['Company ID','Company Name','Industry','SIC Code'])
 .size()
 .reset_index()[['Company ID','Company Name','Industry','SIC Code']]
 .sort_values(by='Company ID'))

,Company ID,Company Name,Industry,SIC Code
0,4976235,E Company,Machinery,5878
1,4977221,I Company,Building Products,5624
2,4986826,L Company,Auto Components,6090
3,498856,J Company,Auto Components,6090
4,4989787,C Company,Machinery,5755
5,4991368,B Company,Auto Components,4937
6,4993687,A Company,Machinery,6085
7,4994208,F Company,Machinery,6085
8,4994275,D Company,Machinery,5722
9,499609,G Company,Auto Components,4150


#### Check Trading Currency

In [70]:
cleaned_data.columns

Index(['Company ID', 'Fiscal Year', 'Industry', 'Trading Currency', 'SP',
       'CDS', 'APD', 'ARD', 'ADA', 'Company Name', 'SIC Code'],
      dtype='object')

In [71]:
cleaned_data[~cleaned_data['Trading Currency'].isin(['USD','GBP'])]

,Company ID,Fiscal Year,Industry,Trading Currency,SP,CDS,APD,ARD,ADA,Company Name,SIC Code
4,4993687,2014,Machinery,3$,55,950,91,34,0,A Company,6085


One record under A Company has issues under currency column. We expect this to be either USD or GBP.

In [72]:
np.unique(cleaned_data[cleaned_data['Company Name']=="A Company"]['Trading Currency'])

array(['3$', 'USD'], dtype=object)

From data, currency associated with A Company is USD. Check in database.

In [73]:
np.unique(Example_DB[Example_DB['Company Name']=="A Company"]['Trading Currency'])

array(['USD'], dtype=object)

All searches pointed at USD, thus the fix we will be applying here is transforming '3$' into 'USD'.

In [74]:
currency_array = cleaned_data['Trading Currency']
currency_array = np.where(currency_array=='3$','USD',currency_array)

In [75]:
cleaned_data['Trading Currency'] = currency_array

In [76]:
np.unique(cleaned_data['Trading Currency'])

array(['GBP', 'USD'], dtype=object)

Checkpoint: save cleaned Example Data in csv

In [77]:
cleaned_data.to_csv('example_data_clean.csv',index=False)

In [108]:
cleaned_data.shape

(111, 11)

### Formatting Data to Match with Database

In [78]:
cleaned_data.columns

Index(['Company ID', 'Fiscal Year', 'Industry', 'Trading Currency', 'SP',
       'CDS', 'APD', 'ARD', 'ADA', 'Company Name', 'SIC Code'],
      dtype='object')

In [79]:
cleaned_data_format = cleaned_data.set_index(['Company ID', 'Company Name', 'Fiscal Year', 'Industry', 'SIC Code','Trading Currency'])
cleaned_data_format = cleaned_data_format.stack(dropna=False).reset_index()
cleaned_data_format.columns = ['Company ID', 'Company Name', 'Fiscal Year', 'Industry', 'SIC Code','Trading Currency', 'Metric Name', 'Value']

In [80]:
cleaned_data_format.columns

Index(['Company ID', 'Company Name', 'Fiscal Year', 'Industry', 'SIC Code',
       'Trading Currency', 'Metric Name', 'Value'],
      dtype='object')

In [81]:
cleaned_data_format = cleaned_data_format.sort_values(by = ['Metric Name','Company Name','Company ID','Industry','Fiscal Year'],ascending=[False,True,True,True,True])

In [82]:
cleaned_data_format.head(10)

,Company ID,Company Name,Fiscal Year,Industry,SIC Code,Trading Currency,Metric Name,Value
0,4993687,A Company,2010,Machinery,6085,USD,SP,75
5,4993687,A Company,2011,Machinery,6085,USD,SP,66
10,4993687,A Company,2012,Machinery,6085,USD,SP,29
15,4993687,A Company,2013,Machinery,6085,USD,SP,56
20,4993687,A Company,2014,Machinery,6085,USD,SP,55
25,4993687,A Company,2015,Machinery,6085,USD,SP,19
30,4993687,A Company,2016,Machinery,6085,USD,SP,35
35,4993687,A Company,2017,Machinery,6085,USD,SP,66
40,4993687,A Company,2018,Machinery,6085,USD,SP,52
45,4991368,B Company,2009,Auto Components,4937,USD,SP,22


In [107]:
cleaned_data_format.shape

(555, 9)

Checkpoint: Formatted clean data

In [83]:
cleaned_data_format.to_csv('example_data_clean_formatted.csv',index=False)

### Comparison

Check consistency of data types before comparison.

In [84]:
Example_DB[['Company ID','Company Name','Industry','SIC Code','Trading Currency','Metric Name']] = Example_DB[['Company ID','Company Name','Industry','SIC Code','Trading Currency','Metric Name']].astype(str)
Example_DB[['Fiscal Year','Value']] = Example_DB[['Fiscal Year','Value']].astype('Int64')

In [85]:
Example_DB.dtypes

Company ID          object
Company Name        object
Fiscal Year          Int64
Industry            object
SIC Code            object
Trading Currency    object
Metric Name         object
Value                Int64
dtype: object

In [86]:
cleaned_data_format.dtypes

Company ID          object
Company Name        object
Fiscal Year          int64
Industry            object
SIC Code            object
Trading Currency    object
Metric Name         object
Value                Int64
dtype: object

Save the columns names for comparison

In [87]:
col_names = list(Example_DB.columns)
col_names

['Company ID',
 'Company Name',
 'Fiscal Year',
 'Industry',
 'SIC Code',
 'Trading Currency',
 'Metric Name',
 'Value']

In [88]:
Example_DB['data_source'] = 'DB'
cleaned_data_format['data_source'] = 'data'

In [89]:
compare = pd.merge(cleaned_data_format,Example_DB,how="outer",on=col_names[:-1], suffixes=('_data', '_db'))

In [90]:
compare.columns

Index(['Company ID', 'Company Name', 'Fiscal Year', 'Industry', 'SIC Code',
       'Trading Currency', 'Metric Name', 'Value_data', 'data_source_data',
       'Value_db', 'data_source_db'],
      dtype='object')

In [91]:
compare.head(10)

,Company ID,Company Name,Fiscal Year,Industry,SIC Code,Trading Currency,Metric Name,Value_data,data_source_data,Value_db,data_source_db
0,4993687,A Company,2010,Machinery,6085,USD,SP,75,data,75,DB
1,4993687,A Company,2011,Machinery,6085,USD,SP,66,data,66,DB
2,4993687,A Company,2012,Machinery,6085,USD,SP,29,data,29,DB
3,4993687,A Company,2013,Machinery,6085,USD,SP,56,data,56,DB
4,4993687,A Company,2014,Machinery,6085,USD,SP,55,data,<NA>,NaN
5,4993687,A Company,2015,Machinery,6085,USD,SP,19,data,19,DB
6,4993687,A Company,2016,Machinery,6085,USD,SP,35,data,35,DB
7,4993687,A Company,2017,Machinery,6085,USD,SP,66,data,66,DB
8,4993687,A Company,2018,Machinery,6085,USD,SP,52,data,52,DB
9,4991368,B Company,2009,Auto Components,4937,USD,SP,22,data,<NA>,NaN


In [92]:
Example_Compare = pd.DataFrame()

#### 1. Values not equal
Expect records to be found in both data sources, but values of matrics do not match.

In [93]:
matched_data = compare[(~compare.data_source_data.isnull())&(~compare.data_source_db.isnull())]

In [94]:
value_not_equal = matched_data[matched_data.Value_data!=matched_data.Value_db][col_names[:-1]+['Value_data','Value_db']]
value_not_equal

,Company ID,Company Name,Fiscal Year,Industry,SIC Code,Trading Currency,Metric Name,Value_data,Value_db
22,4989787,C Company,2013,Machinery,5755,USD,SP,172,221
97,4986826,L Company,2015,Auto Components,6090,USD,SP,179,17
99,4986826,L Company,2017,Auto Components,6090,USD,SP,196,19
133,4989787,C Company,2013,Machinery,5755,USD,CDS,95,52
244,4989787,C Company,2013,Machinery,5755,USD,ARD,91,84


In [95]:
one_is_null = matched_data[(matched_data.Value_data.isnull()&~matched_data.Value_db.isnull())|(~matched_data.Value_data.isnull()&matched_data.Value_db.isnull())][col_names[:-1]+['Value_data','Value_db']]
one_is_null

,Company ID,Company Name,Fiscal Year,Industry,SIC Code,Trading Currency,Metric Name,Value_data,Value_db
141,4994275,D Company,2011,Machinery,5722,USD,CDS,<NA>,0
142,4994275,D Company,2012,Machinery,5722,USD,CDS,<NA>,0


Remaining data here where we can find records on both data file and database are both having null values, which is equal in value.

In [96]:
UnEqual = pd.concat([value_not_equal,one_is_null])
UnEqual['ERROR Type'] = 'UnEqual'

In [97]:
Example_Compare = pd.concat([Example_Compare,UnEqual])

#### 2. Data in file but not in DB

In [98]:
in_file_not_db = compare[(~compare.data_source_data.isnull())&(compare.data_source_db.isnull())][col_names[:-1]+['Value_data','Value_db']]
in_file_not_db['ERROR Type'] = 'Not_in_DB'

In [99]:
Example_Compare = pd.concat([Example_Compare,in_file_not_db])

In [100]:
Example_Compare.shape

(128, 10)

#### 3. Data in DB but not in file

In [101]:
in_db_not_file = compare[(compare.data_source_data.isnull())&(~compare.data_source_db.isnull())][col_names[:-1]+['Value_data','Value_db']]
in_db_not_file['ERROR Type'] = 'Not_in_File'

In [102]:
in_db_not_file

,Company ID,Company Name,Fiscal Year,Industry,SIC Code,Trading Currency,Metric Name,Value_data,Value_db,ERROR Type


We do not observe any values that's in database but not in file. (The example given in the Excel file for Not_in_File error does not exist under database tab)

In [103]:
Example_Compare = pd.concat([Example_Compare,in_db_not_file])

In [104]:
Example_Compare.shape

(128, 10)

In [105]:
Example_Compare.head(10)

,Company ID,Company Name,Fiscal Year,Industry,SIC Code,Trading Currency,Metric Name,Value_data,Value_db,ERROR Type
22,4989787,C Company,2013,Machinery,5755,USD,SP,172,221,UnEqual
97,4986826,L Company,2015,Auto Components,6090,USD,SP,179,17,UnEqual
99,4986826,L Company,2017,Auto Components,6090,USD,SP,196,19,UnEqual
133,4989787,C Company,2013,Machinery,5755,USD,CDS,95,52,UnEqual
244,4989787,C Company,2013,Machinery,5755,USD,ARD,91,84,UnEqual
141,4994275,D Company,2011,Machinery,5722,USD,CDS,<NA>,0,UnEqual
142,4994275,D Company,2012,Machinery,5722,USD,CDS,<NA>,0,UnEqual
4,4993687,A Company,2014,Machinery,6085,USD,SP,55,<NA>,Not_in_DB
9,4991368,B Company,2009,Auto Components,4937,USD,SP,22,<NA>,Not_in_DB
10,4991368,B Company,2010,Auto Components,4937,USD,SP,18,<NA>,Not_in_DB


In [109]:
print(Example_Compare)

    Company ID Company Name  Fiscal Year           Industry SIC Code  \
22     4989787    C Company         2013          Machinery     5755   
97     4986826    L Company         2015    Auto Components     6090   
99     4986826    L Company         2017    Auto Components     6090   
133    4989787    C Company         2013          Machinery     5755   
244    4989787    C Company         2013          Machinery     5755   
..         ...          ...          ...                ...      ...   
500    4976235    E Company         2016          Machinery     5878   
501    4976235    E Company         2017          Machinery     5878   
502    4976235    E Company         2018          Machinery     5878   
523    4977221    I Company         2018  Building Products     5624   
531    4996197    K Company         2015          Machinery     5788   

    Trading Currency Metric Name  Value_data  Value_db ERROR Type  
22               USD          SP         172       221    UnEqual  

Checkpoint: Save the records with errors

In [106]:
Example_Compare.to_csv('example_data_vs_db.csv',index=False)